### Q8 ETH 幣流追蹤

參考 https://hacked.slowmist.io/?c=ETH

追蹤案件:  https://silofinance.medium.com/post-mortem-unreleased-leverage-contract-exploitd-0ab8f37afcbb



簡單來說該案件源於合約漏洞，ETH 合約 Silo protocol 包含漏洞，被強制提領了 224 個 ETH

關鍵交易如下:

https://www.oklink.com/zh-hant/ethereum/tx/0x1f15a193db3f44713d56c4be6679b194f78c2bcdd2ced5b0c7495b7406f5e87a

最終結果如下

- 發送方
    Token: Silo Finance Borrowable WETH Deposit, SiloId: 105
- 接收方
    0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62

轉賬 224 WETH

第一步

先查看詐騙交易的詳細資料, tx: `0x1f15a193db3f44713d56c4be6679b194f78c2bcdd2ced5b0c7495b7406f5e87a`

```sql
DECLARE target_tx STRING DEFAULT LOWER('0x1f15a193db3f44713d56c4be6679b194f78c2bcdd2ced5b0c7495b7406f5e87a');
DECLARE target_date_start DATE DEFAULT DATE('2025-06-24');
DECLARE target_date_end DATE DEFAULT DATE('2025-06-27');

WITH tx_info AS (
    SELECT *
    FROM `bigquery-public-data.crypto_ethereum.transactions`
    WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
      AND `hash` = target_tx
),
tx_logs AS (
    SELECT
        transaction_hash,
        ARRAY_AGG(
            STRUCT(
                log_index,
                address AS contract_address,
                ARRAY_TO_STRING(topics, ' | ') AS topics_string,
                data
            ) ORDER BY log_index ASC
        ) AS all_logs
    FROM `bigquery-public-data.crypto_ethereum.logs`
    WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
      AND transaction_hash = target_tx
    GROUP BY transaction_hash
)

SELECT
    t.*,
    l.all_logs
FROM
    tx_info t
LEFT JOIN
    tx_logs l ON t.`hash` = l.transaction_hash;
```

在以下欄位可以得知黑客地址與黑客觸發攻擊使用的合約地址

- "from_address": "0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62",
- "to_address": "0x79c5c002410a67ac7a0cde2c2217c3f560859c7e",

並且，從 tx_logs 中 index15 的操作可以查到關鍵 event 即:

- silo合約的 Borrow 方法被調用
- silo合約把 224 個 WETH 傳送給黑客 (0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62)

其中，silo合約的地址是: `0x160287e2d3fdcde9e91317982fc1cc01c1f94085`

```json
{
"log_index": "15",
"contract_address": "0x160287e2d3fdcde9e91317982fc1cc01c1f94085",
"topics_string": "0x96558a334f4759f0e7c423d68c84721860bd8fbf94ddc4e55158ecb125ad04b5 | 0x000000000000000000000000cbee4617abf667830fe3ee7dc8d6f46380829df9 | 0x00000000000000000000000004377cfaf4b4a44bb84042218cdda4cebcf8fd62 | 0x00000000000000000000000060baf994f44dd10c19c0c47cbfe6048a4ffe4860",
"data": "0x00000000000000000000000000000000000000000000000c249fdd327780000000000000000000000000000000000000000000000000000c244905318ce21fb8"
,
}
```

data 的解讀如下，可以由 silo code 得知

- asset: 0xc249fdd3277800000 = 224000000000000000000 = 224ETH
- share: 0xc244905318ce21fb8

接著追查該攻擊使用合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e)由誰發布

```sql
DECLARE target_contract STRING DEFAULT LOWER('0x79c5c002410a67ac7a0cde2c2217c3f560859c7e');
DECLARE target_date_start DATE DEFAULT DATE('2025-06-24');
DECLARE target_date_end DATE DEFAULT DATE('2025-06-27');

SELECT
    from_address AS deployer,
    `hash` AS tx_hash,
    block_timestamp
FROM
    `bigquery-public-data.crypto_ethereum.transactions`
WHERE
    DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
    AND receipt_contract_address = target_contract;
```

在欄位 deployer 中可以輕易得知發布者是黑客自己的地址 (0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62)

接著查看黑客地址的交易紀錄，了解其初始資金來源

```sql
DECLARE target_addr STRING DEFAULT LOWER('0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62');
DECLARE target_date_start DATE DEFAULT DATE('2025-06-24');
DECLARE target_date_end DATE DEFAULT DATE('2025-06-27');

-- 1. 找出該地址在指定時間內參與的「所有」交易 Hash 集合
WITH relevant_txs AS (
    SELECT DISTINCT transaction_hash
    FROM (
        -- 情況 A: 該地址出現在主交易中
        SELECT `hash` AS transaction_hash
        FROM `bigquery-public-data.crypto_ethereum.transactions`
        WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
          AND (from_address = target_addr OR to_address = target_addr)

        UNION ALL

        -- 情況 B: 該地址只出現在內部交易 (Inner Transactions / Traces) 中
        SELECT transaction_hash
        FROM `bigquery-public-data.crypto_ethereum.traces`
        WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
          AND (from_address = target_addr OR to_address = target_addr)
    )
),

-- 2. 抓取這些目標交易的「主交易完整資訊」
tx_info AS (
    SELECT t.*
    FROM `bigquery-public-data.crypto_ethereum.transactions` t
    INNER JOIN relevant_txs r ON t.hash = r.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
),

-- 3. 抓取這些目標交易的「內部交易 (Inner Transactions)」，並打包成陣列
tx_traces AS (
    SELECT
        t.transaction_hash,
        ARRAY_AGG(
            STRUCT(
                t.from_address,
                t.to_address,
                t.value / POW(10, 18) AS eth_value, -- 轉為 ETH 方便閱讀
                t.trace_type,   -- 例如: call, create, suicide
                t.call_type,    -- 例如: call, delegatecall
                t.error         -- 如果內部呼叫失敗會有錯誤訊息 (如 Reverted)
            )
        ) AS inner_transactions
    FROM `bigquery-public-data.crypto_ethereum.traces` t
    INNER JOIN relevant_txs r ON t.transaction_hash = r.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
    GROUP BY t.transaction_hash
)

SELECT
    t.*,
    tr.inner_transactions
FROM
    tx_info t
LEFT JOIN
    tx_traces tr ON t.`hash` = tr.transaction_hash
ORDER BY
    t.block_timestamp ASC;
```

查看第一筆交易的 inner_transactions 中兩筆包含 eth 轉入的 tx

```json lines
{
  "from_address": "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc",
  "to_address": "0x94596b6a626392f5d972d6cc4d929a42c2f0008c",
  "eth_value": 0.00452101640095,
  "trace_type": "call",
  "call_type": "call"
}
{
  "from_address": "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc",
  "to_address": "0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62",
  "eth_value": 0.09547898359905,
  "trace_type": "call",
  "call_type": "call"
}
```

可以發現都來自 0.1ETH 的 mixer (0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc)，共轉入 0.1 ETH 作為黑客的初始 gas

此時如果追查該筆 inner transaction 如何觸發，可以查到地址 `0x03392600086874456e08d2bac104380bcdebcfc0`，不過這就是一個遊走在灰色地帶的 relayer，已經脫離鏈上範圍，因此就不深究了。

note:

- 對 relayer 來說，他們只是收到了一份證明，麻煩他們協助提交，
- 提交後黑客就可以拿到錢，所以理論上 relayer 很有機會親自接觸到黑客本人

最終可以整理出以下流程

1. relayer(0x03392600086874456e08d2bac104380bcdebcfc0): 向混幣器(0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc)提出proof
2. 黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62): 的初始資金來自0.1ETH混幣器(0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc)
3. 黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62): 調用合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e)
4. 合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e): 是由黑客(0x04377cfaF4b4A44bb84042218cdDa4cEBCf8fd62)發布
5. 合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e): 操作了silo合約(0x160287e2d3fdcde9e91317982fc1cc01c1f94085)，把224個WETH傳送給黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62)

```txt
【 匿名提資階段 】
   0.1 ETH 混幣器 (Tornado) <---+
   (0x12D6...B8Fc)              |
          |                     | (1) 提交提款 Proof
          | (2) 釋放 0.1 ETH     |
          v               Relayer (中繼者)
       黑客錢包            (0x0339...cfc0)
   (0x0437...fd62)
          |
   -------+---------------------------------------
          |
       【 攻擊執行階段 】
          | (3) 部署合約 (Deploy)
          v
       攻擊合約 (Exploit Contract)
   (0x79c5...9c7e) <----------+
          |                   | (4) 黑客發起調用 (Call)
          | (5) 操作漏洞       |
          v                   |
       Silo 合約 (Victim)      |
   (0x1602...4085)            |
          |                   |
          | (6) 轉出 224 WETH  |
          +-------------------+------> 【 黑客獲利 】
```


補充文件
---

## 隱私協議科普：Tornado Cash 核心原理與黑客洗錢流程

Tornado Cash 是建立在以太坊上的去中心化隱私協議，其核心目標是透過技術手段**切斷發送者與接收者之間的鏈上關聯**。

---

### 一、 Tornado Cash 的核心方法與運作原理

Tornado Cash 的技術基石是 **zk-SNARKs (簡明非互動零知識證明)**。它允許使用者證明「我有權提取某筆資金」，而無需揭露該資金對應的原始存款地址。

#### 運作階段拆解

##### 1. 存款階段 (Deposit)

* **生成憑證：** 使用者在瀏覽器本地隨機生成兩個秘密數值：`Secret` (秘密) 與 `Nullifier` (作廢碼)。
* **計算承諾值 (Commitment)：** 系統將兩者進行哈希運算，得出 `Commitment` 字串。
* **資金上鏈：** 使用者將 ETH 連同 `Commitment` 發送至智能合約。合約鎖定資金並將 Commitment 加入 **默克爾樹 (Merkle Tree)** 中。
* **保存私鑰：** 使用者須自行保存包含上述數值的憑證（Note），這是提款的唯一證明。

##### 2. 提款階段 (Withdraw)

* **生成零知識證明：** 提款時，使用者在本地利用憑證生成 zk-SNARK 證明。此證明僅證實「我知道某個 Commitment 的私鑰」，但不透露是哪一個。
* **提交作廢碼 (Nullifier Hash)：** 提款同時需提交 `Nullifier` 的哈希值。合約會紀錄此值以**防止雙重支付 (Double Spending)**。
* **解除關聯：** 由於提款過程未揭露具體的存款路徑，資金軌跡在鏈上被成功切斷。

---

### 二、 為什麼需要 Relayer（中繼者）？

在提款時，使用者面臨一個**「雞與蛋」的難題**：若要提領到一個**全新地址**以保持匿名，該地址因沒有 ETH 而無法支付 Gas Fee（礦工費）。若從舊地址轉入 Gas Fee，則會暴露連結。

**Relayer (中繼者) 的介入解決了此問題：**

| 功能 | 說明 |
| --- | --- |
| **代付 Gas Fee** | 中繼者是擁有充足 ETH 的第三方節點，專門替使用者支付提款手續費。 |
| **廣播交易** | 使用者將 zk-SNARK 證明與「目標地址」發送給中繼者，由其代為廣播至網路。 |
| **收取手續費** | 作為回報，中繼者會從提領金額中自動扣除比例（約 0.1% ~ 0.5%）作為報酬。 |
| **去信任化** | 提款地址與手續費已寫死在證明中，中繼者**無法竄改地址**也**無法盜取資金**。 |

---

### 三、 常見的駭客使用流程 (Hacker Flow)

駭客在攻擊 DeFi 協議獲利後，通常遵循以下標準作業流程 (SOP) 進行洗錢：

#### 步驟一：資產轉換 (Swapping)

* 將盜取的各種 ERC-20 代幣（如 USDC, USDT）透過 **Uniswap** 等 DEX 全數兌換為 **ETH**。
* 原因：ETH 池子的流動性最大，匿名集 (Anonymity Set) 最廣，混幣效果最佳。

#### 步驟二：分批存款 (Depositing)

* 使用自動化腳本將資金拆分為固定面額（如 100 ETH），分批、分散時間存入合約。

#### 步驟三：沉澱與等待 (Waiting / Aging)

* **關鍵步驟**：資金需在池中「沉澱」數天或數週。
* 目的：等待更多使用者的交易發生，混淆時間線，讓鏈上分析師無法透過存取款時間差進行推斷。

#### 步驟四：透過 Relayer 提款 (Withdrawing via Relayer)

* 創建大量全新的地址。
* 透過 Relayer 機制代付 Gas Fee，將乾淨的 ETH 提領至新地址。

#### 步驟五：變現出金 (Off-ramping)

* 將看似「乾淨」的 ETH 轉移至監管鬆散的中心化交易所 (CEX)。
* 或透過跨鏈橋 (Cross-chain bridge) 轉移至其他公鏈（如 Tron 或 Bitcoin）進行最終套現。


## 資金去向

接著我們來找找資金最後去往何處

查看黑客帳戶(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62)的所有交易

找到轉出財富的 transaction

```sql
DECLARE target_addr STRING DEFAULT LOWER('0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62');
DECLARE target_date_start DATE DEFAULT DATE('2025-06-24');
DECLARE target_date_end DATE DEFAULT DATE('2025-06-27');

-- 1. 找出該地址在指定時間內參與的「所有」交易 Hash 集合
WITH relevant_txs AS (
    SELECT DISTINCT transaction_hash
    FROM (
        -- 情況 A: 該地址出現在主交易中
        SELECT `hash` AS transaction_hash
        FROM `bigquery-public-data.crypto_ethereum.transactions`
        WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
          AND (from_address = target_addr OR to_address = target_addr)

        UNION ALL

        -- 情況 B: 該地址只出現在內部交易 (Inner Transactions / Traces) 中
        SELECT transaction_hash
        FROM `bigquery-public-data.crypto_ethereum.traces`
        WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
          AND (from_address = target_addr OR to_address = target_addr)
    )
),

-- 2. 抓取這些目標交易的「主交易完整資訊」
tx_info AS (
    SELECT t.*
    FROM `bigquery-public-data.crypto_ethereum.transactions` t
    INNER JOIN relevant_txs r ON t.hash = r.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
),

-- 3. 抓取這些目標交易的「內部交易 (Inner Transactions)」，並打包成陣列
tx_traces AS (
    SELECT
        t.transaction_hash,
        ARRAY_AGG(
            STRUCT(
                t.from_address,
                t.to_address,
                t.value / POW(10, 18) AS eth_value, -- 轉為 ETH 方便閱讀
                t.trace_type,   -- 例如: call, create, suicide
                t.call_type,    -- 例如: call, delegatecall
                t.error         -- 如果內部呼叫失敗會有錯誤訊息 (如 Reverted)
            )
        ) AS inner_transactions
    FROM `bigquery-public-data.crypto_ethereum.traces` t
    INNER JOIN relevant_txs r ON t.transaction_hash = r.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
    GROUP BY t.transaction_hash
)

SELECT
    t.*,
    tr.inner_transactions
FROM
    tx_info t
LEFT JOIN
    tx_traces tr ON t.`hash` = tr.transaction_hash
ORDER BY
    t.block_timestamp ASC;
```

在倒數第三筆 transaction 就可以找到價值 224 ETH 的轉出

```json
{
  "hash": "0xc8a21198613a540d12f001228440bb18c30f47a2f5a3d2839b98c782ee7908b7",
  "nonce": "6",
  "transaction_index": "204",
  "from_address": "0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62",
  "to_address": "0x03af609ec30af68e4881126f692c0aec150e84e3",
  "value": "224075307985750903257",
  "gas": "21000",
  "gas_price": "10000000000",
  "input": "0x",
  "receipt_cumulative_gas_used": "12332759",
  "receipt_gas_used": "21000",
  "receipt_contract_address": null,
  "receipt_root": null,
  "receipt_status": "1",
  "block_timestamp": "2025-06-25 14:22:35.000000 UTC",
  "block_number": "22782018",
  "block_hash": "0x08e836c2c8404c46f040deb107ac7bcfdca10d1f37b5351b36c882b60d879487",
  "max_fee_per_gas": "10000000000",
  "max_priority_fee_per_gas": "9000000000",
  "transaction_type": "2",
  "receipt_effective_gas_price": "10000000000",
  "max_fee_per_blob_gas": null,
  "blob_versioned_hashes": [],
  "receipt_blob_gas_price": null,
  "receipt_blob_gas_used": null,
  "inner_transactions": [{
    "from_address": "0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62",
    "to_address": "0x03af609ec30af68e4881126f692c0aec150e84e3",
    "eth_value": "224.07530798575092",
    "trace_type": "call",
    "call_type": "call",
    "error": null
  }]
}
```

看起來被轉入地址 0x03af609ec30af68e4881126f692c0aec150e84e3，所以下一步搜索該地址在時間內的交易

```sql
DECLARE target_addr STRING DEFAULT LOWER('0x03af609ec30af68e4881126f692c0aec150e84e3');
DECLARE target_date_start DATE DEFAULT DATE('2025-06-24');
DECLARE target_date_end DATE DEFAULT DATE('2025-06-27');

-- 1. 找出該地址在指定時間內參與的「所有」交易 Hash 集合
WITH relevant_txs AS (
    SELECT DISTINCT transaction_hash
    FROM (
        -- 情況 A: 該地址出現在主交易中
        SELECT `hash` AS transaction_hash
        FROM `bigquery-public-data.crypto_ethereum.transactions`
        WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
          AND (from_address = target_addr OR to_address = target_addr)

        UNION ALL

        -- 情況 B: 該地址只出現在內部交易 (Inner Transactions / Traces) 中
        SELECT transaction_hash
        FROM `bigquery-public-data.crypto_ethereum.traces`
        WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
          AND (from_address = target_addr OR to_address = target_addr)
    )
),

-- 2. 抓取這些目標交易的「主交易完整資訊」
tx_info AS (
    SELECT t.*
    FROM `bigquery-public-data.crypto_ethereum.transactions` t
    INNER JOIN relevant_txs r ON t.hash = r.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
),

-- 3. 抓取這些目標交易的「內部交易 (Inner Transactions)」，並打包成陣列
tx_traces AS (
    SELECT
        t.transaction_hash,
        ARRAY_AGG(
            STRUCT(
                t.from_address,
                t.to_address,
                t.value / POW(10, 18) AS eth_value, -- 轉為 ETH 方便閱讀
                t.trace_type,   -- 例如: call, create, suicide
                t.call_type,    -- 例如: call, delegatecall
                t.error         -- 如果內部呼叫失敗會有錯誤訊息 (如 Reverted)
            )
        ) AS inner_transactions
    FROM `bigquery-public-data.crypto_ethereum.traces` t
    INNER JOIN relevant_txs r ON t.transaction_hash = r.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
    GROUP BY t.transaction_hash
)

SELECT
    t.*,
    tr.inner_transactions
FROM
    tx_info t
LEFT JOIN
    tx_traces tr ON t.`hash` = tr.transaction_hash
ORDER BY
    t.block_timestamp ASC;
```

很輕易地可以解讀出來，第一筆就是轉入的 224 ETH 贓款

```json
{
  "hash": "0xc8a21198613a540d12f001228440bb18c30f47a2f5a3d2839b98c782ee7908b7",
  "nonce": "6",
  "transaction_index": "204",
  "from_address": "0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62",
  "to_address": "0x03af609ec30af68e4881126f692c0aec150e84e3",
  "value": "224075307985750903257",
  "gas": "21000",
  "gas_price": "10000000000",
  "input": "0x",
  "receipt_cumulative_gas_used": "12332759",
  "receipt_gas_used": "21000",
  "receipt_contract_address": null,
  "receipt_root": null,
  "receipt_status": "1",
  "block_timestamp": "2025-06-25 14:22:35.000000 UTC",
  "block_number": "22782018",
  "block_hash": "0x08e836c2c8404c46f040deb107ac7bcfdca10d1f37b5351b36c882b60d879487",
  "max_fee_per_gas": "10000000000",
  "max_priority_fee_per_gas": "9000000000",
  "transaction_type": "2",
  "receipt_effective_gas_price": "10000000000",
  "max_fee_per_blob_gas": null,
  "blob_versioned_hashes": [],
  "receipt_blob_gas_price": null,
  "receipt_blob_gas_used": null,
  "inner_transactions": [{
    "from_address": "0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62",
    "to_address": "0x03af609ec30af68e4881126f692c0aec150e84e3",
    "eth_value": "224.07530798575092",
    "trace_type": "call",
    "call_type": "call",
    "error": null
  }]
}
```

接著關注比較大筆的交易，基本上就是把 224 分拆存入不同 Mixer 中

例如下面就是把 100 ETH 存入 Mixer 的交易

```json
{
  "hash": "0xfac5be1f87d21052bec0b195d73cbab8c09281c23a4e52173fe616f4624800c4",
  "nonce": "1",
  "transaction_index": "0",
  "from_address": "0x03af609ec30af68e4881126f692c0aec150e84e3",
  "to_address": "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b",
  "value": "100000000000000000000",
  "gas": "1018645",
  "gas_price": "10000000000",
  "input": "0x13d98d13000000000000000000000000a160cdab225685da1d56aa342ad8841c3b53f291075ee19bf4246fb29117647ec6637d7dd98c6331ef29c760aee847c5e3f697a700000000000000000000000000000000000000000000000000000000000000600000000000000000000000000000000000000000000000000000000000000000",
  "receipt_cumulative_gas_used": "914236",
  "receipt_gas_used": "914236",
  "receipt_contract_address": null,
  "receipt_root": null,
  "receipt_status": "1",
  "block_timestamp": "2025-06-26 18:25:35.000000 UTC",
  "block_number": "22790390",
  "block_hash": "0xe39a22bda9fcbfe938ebf0c39c308c00bec82d1d2c8e700d801650dcda3455b7",
  "max_fee_per_gas": "10000000000",
  "max_priority_fee_per_gas": "9000000000",
  "transaction_type": "2",
  "receipt_effective_gas_price": "10000000000",
  "max_fee_per_blob_gas": null,
  "blob_versioned_hashes": [],
  "receipt_blob_gas_price": null,
  "receipt_blob_gas_used": null,
}
```

這邊有一個有趣的現象，同一個 mixer contract 其實是一個 router contract

會根據傳入的金額和其他條件，把 ETH rout 到不同的資金池合約

如果想追蹤的金額是 0.1 或 1 ETH ，因為金額小，常發生，所以每個資金池的金額進出頻繁，難以追蹤

但是 100 ETH 金額太大，比較少發生，所以查看短期資金池輸出的特徵，或許有機會抓到黑客的漏洞

```json
{
    "from_address": "0xa160cdab225685da1d56aa342ad8841c3b53f291",
    "to_address": "0x83584f83f26af4edda9cbe8c730bc87c364b28fe",
    "eth_value": "100.0",
    "trace_type": "call",
    "call_type": "delegatecall",
    "error": null
 }
```

從存入 100ETH 的 tx 中找到 inner_transactions，其中 0xa160cdab225685da1d56aa342ad8841c3b53f291 就是 100 ETH 所在的資金池

如果真的去查會發現，0xa160cdab225685da1d56aa342ad8841c3b53f291 確實是 Tornado Cash 治理的資金池

這是個可能的破口，我們可以找尋在相近時間的連續兩筆 100 ETH 提款 (因金額大，發生次數會比較少)，嘗試猜測攻擊者的乾淨錢包

甚至可以在多個池交叉比對可疑地址，...等等。

```sql
DECLARE target_addr STRING DEFAULT LOWER('0xa160cdab225685da1d56aa342ad8841c3b53f291');
DECLARE draw_back_method DEFAULT LOWER('0xe9e508bad6d4c3227e881ca19068f099da81b5164dd6d62b2eaf1e8bc6c34931');
DECLARE target_date_start DATE DEFAULT DATE('2025-06-26');
DECLARE target_date_end DATE DEFAULT DATE('2025-06-28');

-- 1. 透過 Logs 表，找出該合約的所有提款 log，記下 init txid
WITH verified_txs AS (
    SELECT DISTINCT transaction_hash
    FROM `bigquery-public-data.crypto_ethereum.logs`
    WHERE DATE(block_timestamp) BETWEEN target_date_start AND target_date_end
      AND address = target_addr
      AND topics[SAFE_OFFSET(0)] = draw_back_method
),

-- 2. 利用這些確認過的 init txid，去 traces 表抓取「實質的 ETH 轉移」
eth_transfers AS (
    SELECT
        t.transaction_hash,
        t.from_address AS actual_sender_contract,
        t.to_address AS recipient_address,
        t.value / POW(10, 18) AS eth_value
    FROM `bigquery-public-data.crypto_ethereum.traces` t
    INNER JOIN verified_txs v ON t.transaction_hash = v.transaction_hash
    WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
      AND t.value > 0
      AND t.error IS NULL
),

-- 3. 將同一筆交易中的所有 ETH 轉移打包與加總
tx_traces_summary AS (
    SELECT
        transaction_hash,
        SUM(eth_value) AS total_eth_withdrawn,
        ARRAY_AGG(
            STRUCT(
                actual_sender_contract,
                recipient_address,
                eth_value
            )
        ) AS transfer_details
    FROM eth_transfers
    GROUP BY transaction_hash
)

-- 4. 組合主交易與內部交易分析結果
SELECT
    t.hash AS transaction_hash,
    t.block_timestamp,
    t.from_address AS tx_initiator,
    t.to_address AS interacted_contract,
    tr.total_eth_withdrawn,
    tr.transfer_details
FROM `bigquery-public-data.crypto_ethereum.transactions` t
INNER JOIN tx_traces_summary tr ON t.hash = tr.transaction_hash
WHERE DATE(t.block_timestamp) BETWEEN target_date_start AND target_date_end
ORDER BY t.block_timestamp ASC;
```

以上搜索查詢了短期內該資金池的提領，以下僅為舉例，作為教學目的，並非真實確認

硬要找，我個人就會覺得以下這兩筆交易比較可疑

```json lines
{
  "transaction_hash": "0x91bbd76e931c9540848a09069c9e916611eec6233580f62a07e2e8aa4cf72423",
  "block_timestamp": "2025-06-28 00:01:23.000000 UTC",
  "tx_initiator": "0x7d2a66f05d2bb0ff4dc856eb317196c59ac3e932",
  "interacted_contract": "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b",
  "total_eth_withdrawn": "100.0",
  "transfer_details": [{
    "actual_sender_contract": "0xa160cdab225685da1d56aa342ad8841c3b53f291",
    "recipient_address": "0x5b5de62e2b547d42aa16954210fa0cbe4feeda8f",
    "eth_value": "99.598167413255"
  }, {
    "actual_sender_contract": "0xa160cdab225685da1d56aa342ad8841c3b53f291",
    "recipient_address": "0x7d2a66f05d2bb0ff4dc856eb317196c59ac3e932",
    "eth_value": "0.401832586745"
  }]
}
{
  "transaction_hash": "0x5dc46c0ddc16ffa3a28c1d9cc28798eca81b751b65e5e97d923dd48bcde1bc8f",
  "block_timestamp": "2025-06-28 00:02:23.000000 UTC",
  "tx_initiator": "0x7d2a66f05d2bb0ff4dc856eb317196c59ac3e932",
  "interacted_contract": "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b",
  "total_eth_withdrawn": "100.0",
  "transfer_details": [{
    "actual_sender_contract": "0xa160cdab225685da1d56aa342ad8841c3b53f291",
    "recipient_address": "0x5b5de62e2b547d42aa16954210fa0cbe4feeda8f",
    "eth_value": "99.5981596523777"
  }, {
    "actual_sender_contract": "0xa160cdab225685da1d56aa342ad8841c3b53f291",
    "recipient_address": "0x7d2a66f05d2bb0ff4dc856eb317196c59ac3e932",
    "eth_value": "0.40184034762230003"
  }]
}
```

我列出可疑之處

- tx_initiator: 提款者相同
- block_timestamp: 時間相近
- recipient_address: relayer/收款者 相同
- 該地址只完成這件事就不再使用了

最重要的是如果查詢收款者地址 0x5b5de62e2b547d42aa16954210fa0cbe4feeda8f

會發現他在 6/28 總共從 mixer 拿了 230ETH，並且在短時間內把所有資產交易出去

當然，也存在可以反駁之處

最重要的是贓款是 224，這個帳戶卻取了 230

對黑客來說寧可少拿錢，讓錢困在 mixer，也不會想辦法湊整數，因此增加風險。

## 結論

以上我演練了一個完整的幣流追蹤流程，以下我統整成 mermaid flow chart 便於讀者理解

1. relayer(0x03392600086874456e08d2bac104380bcdebcfc0): 向混幣器(0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc)提出proof
2. 黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62): 的初始資金來自0.1ETH混幣器(0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc)
3. 黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62): 調用合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e)
4. 合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e): 是由黑客(0x04377cfaF4b4A44bb84042218cdDa4cEBCf8fd62)發布
5. 合約(0x79c5c002410a67ac7a0cde2c2217c3f560859c7e): 操作了silo合約(0x160287e2d3fdcde9e91317982fc1cc01c1f94085)，把224個WETH傳送給黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62)
6. 黑客(0x04377cfaf4b4a44bb84042218cdda4cebcf8fd62): 把錢轉給黑客帳戶2(0x03af609ec30af68e4881126f692c0aec150e84e3)
7. 黑客帳戶2(0x03af609ec30af68e4881126f692c0aec150e84e3): 把錢存入混幣器合約(0xd90e2f925DA726b50C4Ed8D0Fb90Ad053324F31b)
8. 混幣器合約(0xd90e2f925DA726b50C4Ed8D0Fb90Ad053324F31b): 調用100ETH資金池合約(0xA160cdAB225685dA1d56aa342Ad8841c3b53f291)
9. 混幣器合約(0xd90e2f925DA726b50C4Ed8D0Fb90Ad053324F31b): 調用10ETH資金池合約(0x910Cbd523D972eb0a6f4cAe4618aD62622b39DbF)
10. 混幣器合約(0xd90e2f925DA726b50C4Ed8D0Fb90Ad053324F31b): 調用1ETH資金池合約(0x47CE0C6eD5B0Ce3d3A51fdb1C52DC66a7c3c2936)
11. 混幣器合約(0xd90e2f925DA726b50C4Ed8D0Fb90Ad053324F31b): 調用0.1ETH資金池合約(0x12D66f87A04A9E220743712cE6d9bB1B5616B8Fc)
12. 嫌疑人1(0x5b5dE62e2b547d42AA16954210fA0cBE4FEeda8f): 從 100ETH資金池 取 200
13. 嫌疑人1(0x5b5dE62e2b547d42AA16954210fA0cBE4FEeda8f): 從 10ETH資金池 取 30
14. 嫌疑人1(0x5b5dE62e2b547d42AA16954210fA0cBE4FEeda8f): 利用 swap 合約交易掉手上的所有 ETH

ref: [graph](./Q8.md)